# Missing Value Treatment

## Numerical data

**Missing Value Check for numerical variables**

In [30]:
missing_val_chk(data[numerical_vars])

The missing values summary


,# missing,percent
DurationOfPitch,251,5.135
MonthlyIncome,233,4.767
Age,226,4.624
NumberOfTrips,140,2.864


Since the missing value % across the numerical variables are **~=< 5%**, let's impute the **median** of each variable in the missing values

Using the **SKLearn Simple Imputer** with **median** strategy

In [31]:
# Declare numerical imputer function with median strategy
numerical_imputer = SimpleImputer(missing_values=np.nan, strategy='median')

# Execute numerical imputer function on numerical variables
data[numerical_vars] = numerical_imputer.fit_transform(data[numerical_vars])

# Confirm if there are any missing values after impution
missing_val_chk(data[numerical_vars])

There are NO missing values in the dataset


## Categorical data

**Missing Value Check for categorical variables**

In [32]:
missing_val_chk(data[categorical_vars])

The missing values summary


,# missing,percent
NumberOfChildrenVisited,66,1.350
NumberOfFollowups,45,0.921
PreferredPropertyStar,26,0.532
PreferredLoginDevice,25,0.511


Since the missing value % across the categorical variables are **~=< 1%**, let's impute the **'most_frequent'** of each variable in the missing values

Using the **SKLearn Simple Imputer** with **median** strategy

In [33]:
# Declare categorical imputer function with most_frequent strategy
categorical_imputer = SimpleImputer(missing_values=np.nan, strategy='most_frequent')

# Execute categorical imputer function on categorical variables
data[categorical_vars] = categorical_imputer.fit_transform(data[categorical_vars])

# Confirm if there are any missing values after impution
missing_val_chk(data[categorical_vars])

There are NO missing values in the dataset


---

# Outlier Treatment

In the numerical data analysis, we have observed that`DurationOfPitch`, `NumberOfTrips` & `MonthlyIncome` variables have outliers.

According to literature, **Bagging** Ensemble methods are robust to outliers while **Boosting** Ensemble methods may be sensitive.  
Therefore outlier treatment will only be necessary for the **Boosting** Ensemble methods.

Before we decide on whether or not Outlier Treatment be done prior EDA, lets explore the variables.

<!-- Therefore in further analysis, I shall run the Bagging methods with the data as such. For the Boosting methods, I will clone the data, deal with the outliers the data and compare model performances with & without outliers. -->

---

## Outlier exploration

In [34]:
#Applying the Outlier check function for the sub-dataframe of numerical variables
outlier_count(data[numerical_vars])

The 'DurationOfPitch' distribution has '0' lower outliers and '112' upper outliers.

The 'NumberOfTrips' distribution has '0' lower outliers and '109' upper outliers.

The 'MonthlyIncome' distribution has '2' lower outliers and '373' upper outliers.



In [35]:
def quantile_check(variable):
    """
    This function to explores the variable values at
    predefined quantile intervals.
    This is will aid in determining capping limits.
    """
    quantile_range = [0.8, 0.9, 0.95, 0.98, 0.99, 0.995, 1.0]
    for quantile in quantile_range:
        interval = variable.quantile(quantile)
        print(f"The '{quantile}' quantile value is '{interval}'.")

---

### DurationOfPitch

In [36]:
data.DurationOfPitch.describe()

count    4888.000000
mean       15.362930
std         8.316166
min         5.000000
25%         9.000000
50%        13.000000
75%        19.000000
max       127.000000
Name: DurationOfPitch, dtype: float64

Between Q3 and Q4, there is significant change compared to (Q1-Q2) & (Q2-Q3).  
Let's explore further to see where there is a rapid jump.

In [37]:
quantile_check(data.DurationOfPitch)

The '0.8' quantile value is '22.0'.
The '0.9' quantile value is '29.0'.
The '0.95' quantile value is '32.0'.
The '0.98' quantile value is '35.0'.
The '0.99' quantile value is '35.0'.
The '0.995' quantile value is '36.0'.
The '1.0' quantile value is '127.0'.


There is a rapid jump after the 99.5 percentile.
Let's explore these rows

In [38]:
data[data.DurationOfPitch>data.DurationOfPitch.quantile(0.995)]

,ProdTaken,Age,PreferredLoginDevice,CityTier,DurationOfPitch,Occupation,Gender,NumberOfPersonVisited,NumberOfFollowups,ProductPitched,PreferredPropertyStar,MaritalStatus,NumberOfTrips,Passport,PitchSatisfactionScore,OwnCar,NumberOfChildrenVisited,Designation,MonthlyIncome
1434,0,36.0,Company Invited,3,126.0,Salaried,Male,2,3.0,Multi,3.0,Married,3.0,0,1,1,1.0,Executive,18482.0
3878,0,53.0,Company Invited,3,127.0,Salaried,Male,3,4.0,Multi,3.0,Married,4.0,0,1,1,2.0,Executive,22160.0


Observation:
* There are 2 Married Male customers, who are both 'Executive' of significantly different Age groups

Since there are only *2* rows and the difference in value between the 99.5 and 100 percentiles is very large, let's cap the values at the 99.5 percentile

In [39]:
# capping outliers at 99.5 percentile

data.DurationOfPitch = np.where(
    data.DurationOfPitch > data.DurationOfPitch.quantile(0.995),
    data.DurationOfPitch.quantile(0.995), data.DurationOfPitch)

---

### NumberOfTrips

In [40]:
data.NumberOfTrips.describe()

count    4888.000000
mean        3.229746
std         1.822769
min         1.000000
25%         2.000000
50%         3.000000
75%         4.000000
max        22.000000
Name: NumberOfTrips, dtype: float64

Between Q3 and Q4, there is significant change compared to (Q1-Q2) & (Q2-Q3).  
Let's explore further to see where there is a rapid jump.

In [41]:
quantile_check(data.NumberOfTrips)

The '0.8' quantile value is '5.0'.
The '0.9' quantile value is '6.0'.
The '0.95' quantile value is '7.0'.
The '0.98' quantile value is '8.0'.
The '0.99' quantile value is '8.0'.
The '0.995' quantile value is '8.0'.
The '1.0' quantile value is '22.0'.


There is a rapid jump after the 99.5 percentile.
Let's explore these rows

In [42]:
data[data.NumberOfTrips>data.NumberOfTrips.quantile(0.995)]

,ProdTaken,Age,PreferredLoginDevice,CityTier,DurationOfPitch,Occupation,Gender,NumberOfPersonVisited,NumberOfFollowups,ProductPitched,PreferredPropertyStar,MaritalStatus,NumberOfTrips,Passport,PitchSatisfactionScore,OwnCar,NumberOfChildrenVisited,Designation,MonthlyIncome
385,1,30.0,Company Invited,1,10.0,Large Business,Male,2,3.0,Multi,3.0,Single,19.0,1,4,1,1.0,Executive,17285.0
816,0,39.0,Company Invited,1,15.0,Salaried,Male,3,3.0,Super Deluxe,4.0,Unmarried,21.0,0,2,1,0.0,Manager,21782.0
2829,1,31.0,Company Invited,1,11.0,Large Business,Male,3,4.0,Multi,3.0,Single,20.0,1,4,1,2.0,Executive,20963.0
3260,0,40.0,Company Invited,1,16.0,Salaried,Male,4,4.0,Super Deluxe,4.0,Unmarried,22.0,0,2,1,1.0,Manager,25460.0


Observation:
* There are 4 Male customers, who are 'Executive' & 'Manager' of similar Age groups. They are 'Single' and 'Unmarried'

Since there are only *4* rows and the difference in value between the 99.5 and 100 percentiles is very large, let's cap the values at the 99.5 percentile

In [43]:
# capping outliers at 99.5 percentile

data.NumberOfTrips = np.where(
    data.NumberOfTrips > data.NumberOfTrips.quantile(0.995),
    data.NumberOfTrips.quantile(0.995), data.NumberOfTrips)

---

### MonthlyIncome

In [44]:
data.MonthlyIncome.describe()

count     4888.000000
mean     23559.179419
std       5257.862921
min       1000.000000
25%      20485.000000
50%      22347.000000
75%      25424.750000
max      98678.000000
Name: MonthlyIncome, dtype: float64

Between (Q0 - Q1) & (Q3 - Q4), there is significant change compared to (Q1-Q2) & (Q2-Q3).  
Let's explore further to see where there is a rapid jump.

**Lower Outliers**

In [45]:
def lower_quantile_check(variable):
    """
    This function to explores the variable values at
    predefined quantile intervals.
    This is will aid in determining capping limits.
    """
    quantile_range = [0, 0.05, 0.1, 0.15, 0.2, 0.25]
    for quantile in quantile_range:
        interval = variable.quantile(quantile)
        print(f"The '{quantile}' quantile value is '{interval}'.")

In [46]:
lower_quantile_check(data.MonthlyIncome)

The '0' quantile value is '1000.0'.
The '0.05' quantile value is '17311.7'.
The '0.1' quantile value is '17686.0'.
The '0.15' quantile value is '18291.149999999998'.
The '0.2' quantile value is '19821.0'.
The '0.25' quantile value is '20485.0'.


There is a rapid jump between the 0 and 5 percentile.
Let's explore these rows

In [47]:
data[data.MonthlyIncome<data.MonthlyIncome.quantile(0.05)]

,ProdTaken,Age,PreferredLoginDevice,CityTier,DurationOfPitch,Occupation,Gender,NumberOfPersonVisited,NumberOfFollowups,ProductPitched,PreferredPropertyStar,MaritalStatus,NumberOfTrips,Passport,PitchSatisfactionScore,OwnCar,NumberOfChildrenVisited,Designation,MonthlyIncome
2,1,37.0,Self Enquiry,1,8.0,Free Lancer,Male,3,4.0,Multi,3.0,Single,7.0,1,3,0,0.0,Executive,17090.0
14,1,28.0,Self Enquiry,1,30.0,Salaried,Male,2,4.0,Multi,3.0,Single,6.0,1,2,0,0.0,Executive,17028.0
23,0,21.0,Self Enquiry,1,21.0,Salaried,Male,3,3.0,Multi,3.0,Single,2.0,0,3,1,1.0,Executive,16232.0
32,0,30.0,Self Enquiry,1,15.0,Small Business,Male,2,4.0,Multi,3.0,Single,2.0,0,4,1,1.0,Executive,17206.0
33,1,39.0,Self Enquiry,3,11.0,Large Business,Male,2,3.0,Super Deluxe,3.0,Divorced,4.0,0,2,0,1.0,Manager,17086.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2403,1,28.0,Self Enquiry,1,7.0,Large Business,Male,3,4.0,Multi,5.0,Single,7.0,0,3,1,1.0,Executive,17080.0
2404,1,25.0,Self Enquiry,1,15.0,Salaried,Male,2,3.0,Multi,5.0,Single,4.0,0,1,1,0.0,Executive,17096.0
2422,1,31.0,Self Enquiry,3,19.0,Small Business,Male,3,4.0,Super Deluxe,5.0,Married,6.0,1,1,1,0.0,Manager,17302.0
2442,1,18.0,Self Enquiry,3,15.0,Small Business,Male,2,3.0,Multi,3.0,Single,2.0,0,5,0,1.0,Executive,16611.0


Observation:
* There are many rows where the `MonthlyIncome` is less than the 5 percentile but there some rows where the monthly income is less than 15000.

In [48]:
# Check for rows where `MonthlyIncome` is less than 15000
data[data.MonthlyIncome<15000]

,ProdTaken,Age,PreferredLoginDevice,CityTier,DurationOfPitch,Occupation,Gender,NumberOfPersonVisited,NumberOfFollowups,ProductPitched,PreferredPropertyStar,MaritalStatus,NumberOfTrips,Passport,PitchSatisfactionScore,OwnCar,NumberOfChildrenVisited,Designation,MonthlyIncome
142,0,38.0,Self Enquiry,1,9.0,Large Business,Female,2,3.0,Super Deluxe,3.0,Single,4.0,1,5,0,0.0,Manager,1000.0
2586,0,39.0,Self Enquiry,1,10.0,Large Business,Female,3,4.0,Super Deluxe,3.0,Single,5.0,1,5,0,1.0,Manager,4678.0


There are 2 Female customers, who are similiar in age, 'Manager', 'Single' & have 'MonthlyIncome' less than 15000.  
Let's cap the lowest 'MonthlyIncome' values at 15000 as the presence of such small outliers will skew the data.

In [49]:
# capping outliers ('MonthlyIncome' less than 15000) at 15000

data.MonthlyIncome = np.where(data.MonthlyIncome < 15000, 15000,
                              data.MonthlyIncome)

---

**Upper Outliers**

In [50]:
quantile_check(data.MonthlyIncome)

The '0.8' quantile value is '26867.0'.
The '0.9' quantile value is '31869.9'.
The '0.95' quantile value is '34632.85'.
The '0.98' quantile value is '37418.0'.
The '0.99' quantile value is '38084.0'.
The '0.995' quantile value is '38310.08499999999'.
The '1.0' quantile value is '98678.0'.


There is a rapid jump after the 99.5 percentile.
Let's explore these rows

In [51]:
data[data.MonthlyIncome>data.MonthlyIncome.quantile(0.995)]

,ProdTaken,Age,PreferredLoginDevice,CityTier,DurationOfPitch,Occupation,Gender,NumberOfPersonVisited,NumberOfFollowups,ProductPitched,PreferredPropertyStar,MaritalStatus,NumberOfTrips,Passport,PitchSatisfactionScore,OwnCar,NumberOfChildrenVisited,Designation,MonthlyIncome
38,0,36.0,Self Enquiry,1,11.0,Salaried,Female,2,4.0,Multi,3.0,Divorced,1.0,1,2,1,0.0,Executive,95000.0
2482,0,37.0,Self Enquiry,1,12.0,Salaried,Female,3,5.0,Multi,5.0,Divorced,2.0,1,2,1,1.0,Executive,98678.0
2609,0,51.0,Self Enquiry,1,18.0,Salaried,Female,3,4.0,King,3.0,Single,5.0,0,5,1,1.0,VP,38604.0
2634,0,53.0,Self Enquiry,1,7.0,Salaried,Male,4,5.0,King,3.0,Divorced,2.0,0,2,1,2.0,VP,38677.0
3012,1,56.0,Self Enquiry,1,9.0,Small Business,Male,4,4.0,King,3.0,Divorced,7.0,1,2,1,3.0,VP,38537.0
3190,0,42.0,Company Invited,1,14.0,Salaried,Female,3,6.0,King,3.0,Married,3.0,0,4,1,1.0,VP,38651.0
3193,1,53.0,Self Enquiry,3,9.0,Small Business,Female,3,6.0,King,3.0,Divorced,3.0,0,3,1,1.0,VP,38523.0
3295,0,57.0,Self Enquiry,1,11.0,Large Business,Female,4,4.0,King,3.0,Married,6.0,0,4,0,3.0,VP,38621.0
3342,0,44.0,Self Enquiry,1,10.0,Salaried,Male,4,6.0,King,3.0,Divorced,5.0,0,5,1,3.0,VP,38418.0
3362,0,52.0,Company Invited,3,16.0,Salaried,Male,3,4.0,King,3.0,Married,6.0,1,4,1,2.0,VP,38525.0


Observation:
* There are many rows where the `MonthlyIncome` is greater than the 99.5 percentile but there some rows where the monthly income is greater than 40000.

In [52]:
# Check for rows where `MonthlyIncome` is greater than 40000
data[data.MonthlyIncome>40000]

,ProdTaken,Age,PreferredLoginDevice,CityTier,DurationOfPitch,Occupation,Gender,NumberOfPersonVisited,NumberOfFollowups,ProductPitched,PreferredPropertyStar,MaritalStatus,NumberOfTrips,Passport,PitchSatisfactionScore,OwnCar,NumberOfChildrenVisited,Designation,MonthlyIncome
38,0,36.0,Self Enquiry,1,11.0,Salaried,Female,2,4.0,Multi,3.0,Divorced,1.0,1,2,1,0.0,Executive,95000.0
2482,0,37.0,Self Enquiry,1,12.0,Salaried,Female,3,5.0,Multi,5.0,Divorced,2.0,1,2,1,1.0,Executive,98678.0


There are 2 Female customers, who are similiar in age, 'Executive', 'Divorced' & have 'MonthlyIncome' greater than 90000.  
Let's cap the upper 'MonthlyIncome' values at 40000 as the presence of such large outliers will skew the data.

In [53]:
# capping outliers ('MonthlyIncome' greater than 90000) at 40000

data.MonthlyIncome = np.where(data.MonthlyIncome > 40000, 40000,
                              data.MonthlyIncome)

---